In [21]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import CPTAC.Colon as c

You have loaded the CPTAC Colon dataset. To view available dataframes,
use CPTAC.Colon.list_data(). To view available functions for accessing
and manipulating the dataframes, use CPTAC.Colon.list_api().
Colon Data Version: Most recent release

Loading CPTAC Colon data:
Loading clinical data...
Loading miRNA data...
Loading mutation data...
Loading mutation_binary data...
Loading phosphoproteomics_normal data...
Loading phosphoproteomics_tumor data...
Loading proteomics_normal data...
Loading proteomics_tumor data...
Loading transcriptomics data...


In [92]:
def get_frequently_mutated(cancer_type, cutoff=.1):  

    #import CPTAC 
    if cancer_type == "endometrial" or cancer_type == "Endometrial":
        import CPTAC.Endometrial as CPTAC
        
    elif cancer_type == "colon" or cancer_type == "Colon":
        import CPTAC.Colon as CPTAC
        
    elif cancer_type == "ovarian" or cancer_type == "Ovarian":
        import CPTAC.Ovarian as CPTAC
    
    else:
        print("Please enter a valid cancer type.")
        
    gene_and_freq_d = {}  
        
    # get data frames
    somatic = CPTAC.get_mutations()
    proteomics = CPTAC.get_proteomics()
    gene = 'PTEN'
    omics_mutations = CPTAC.append_mutations_to_omics(mutation_genes=gene, omics_df=prot, omics_genes=gene)
    gene_mutated = somatic.loc[somatic['Gene'] == gene]
    
    # unique genes
    unique_genes = somatic['Gene'].unique()
    
    # get total tumor patients
    tumors = omics_mutations.loc[omics_mutations['Sample_Status'] == 'Tumor']
    total_tumor_patients = len(tumors)
        
    #find frequently mutated
    if gene_mutated.columns[0] == 'Gene':
        for gene in unique_genes:
            gene_mutated = somatic.loc[somatic['Gene'] == gene].index
            num_gene_mutated = len(gene_mutated.unique())
            percentage = (num_gene_mutated / total_tumor_patients)
            if percentage > cutoff:
                gene_and_freq_d[gene] = percentage
    
    print(gene_and_freq_d)
    # create dataframe
    freq_mutated_df = pd.DataFrame(gene_and_freq_d.items())
                   
    return freq_mutated_df

In [95]:
dataf = get_frequently_mutated("colon", .25)

{'APC': 0.845360824742268, 'TP53': 0.5773195876288659, 'MUC16': 0.4020618556701031, 'FAT3': 0.28865979381443296, 'SYNE1': 0.29896907216494845, 'TTN': 0.6185567010309279, 'CSMD3': 0.26804123711340205, 'KRAS': 0.36082474226804123, 'OBSCN': 0.29896907216494845, 'MUC5B': 0.27835051546391754, 'ZNF469': 0.25773195876288657, 'RYR1': 0.25773195876288657, 'ACVR2A': 0.25773195876288657, 'FAT4': 0.26804123711340205, 'CCDC168': 0.29896907216494845, 'DNAH5': 0.26804123711340205}


In [96]:
dataf

,0,1
0,APC,0.845361
1,TP53,0.577320
2,MUC16,0.402062
3,FAT3,0.288660
4,SYNE1,0.298969
5,TTN,0.618557
6,CSMD3,0.268041
7,KRAS,0.360825
8,OBSCN,0.298969
9,MUC5B,0.278351


In [75]:
d = {'TTN': 0.30097087378640774, 'TP53': 0.7475728155339806, 'MUC4': 0.3300970873786408}

In [91]:
df = pd.DataFrame(d.items())
df


,0,1
0,TTN,0.300971
1,TP53,0.747573
2,MUC4,0.330097


In [16]:
#find frequently mutated
unique_genes = somatic['Gene'].unique()
gene_mutated = somatic.loc[somatic['Gene'] == 'PTEN']
gene_and_freq_d = {}
total_tumor_patients = 97
cutoff = .25
for gene in unique_genes:
    gene_mutated = somatic.loc[somatic['Gene'] == gene].iloc[:, 0]
    gene_mutated.drop_duplicates(keep='first',inplace=True)
    num_gene_mutated = len(gene_mutated)
    percentage = (num_gene_mutated / total_tumor_patients)
    if percentage > cutoff:
        gene_and_freq_d[gene] = percentage